In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [ ]:
N=100
epochs=10
mini_batch_size = 20

In [ ]:
train_input, train_target, train_classes, \
test_input, test_target, test_classes = \
    prologue.generate_pair_sets(N)

In [ ]:
#train_classes = nn.functional.one_hot(train_classes, num_classes=10)
#test_classes = nn.functional.one_hot(test_classes, num_classes=10)

In [ ]:
#code from practical 4b
def leq(x,y):
    return x<=y

class Net(nn.Module):
    def __init__(self, nb_hidden):
        super().__init__()
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 20, kernel_size=3),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(20, 50, kernel_size=3),
            nn.MaxPool2d(2, stride=2))

        self.fc1 = nn.Sequential(
            nn.Linear(50 * 2 * 2, 100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 10))
        
    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output
    
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2
    
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs = 25):
    #classification
    criterion = nn.CrossEntropyLoss()
    eta=1e-1
            
    for e in range(nb_epochs):
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                           train_input.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
            
            loss = criterion(output[0], train_target.narrow(0, b, mini_batch_size)[:,0])

            model.zero_grad()
            loss.backward()

            #loss second entry
            loss = criterion(output[1], train_target.narrow(0, b, mini_batch_size)[:,1])
            
            model.zero_grad()
            loss.backward()
            
            print(b)
        print(e)
        
def compute_nb_errors(model, input_, target_, mini_batch_size):
    nb_error = 0
    
    for b in range(0, input_.size(0), mini_batch_size):
        output = model(input_.narrow(0, b, mini_batch_size)[:,0,:].unsqueeze(1),
                       input_.narrow(0, b, mini_batch_size)[:,1,:].unsqueeze(1))
        _,pred_class_0 = output[0].max(1)
        _,pred_class_1 = output[1].max(1)

        for k in range(mini_batch_size):
            if target_[b+k,0] != pred_class_0[k]:
                nb_error += 1
            if target_[b+k,1] != pred_class_1[k]:
                nb_error += 1
                
    return nb_error

# weight sharing -> siamese network 

In [ ]:
model = Net(20)

In [ ]:
train_model(model, train_input, train_classes, mini_batch_size)

In [ ]:
nb_test_errors = compute_nb_errors(model, test_input, test_classes, mini_batch_size)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                  nb_test_errors, test_input.size(0)))